## Azure AI Foundry - Hosted Agents v2

For the time being, install Preview version of Foundry SDK to use Agents v2 API:

``` bash
pip install azure-ai-projects --pre
```

### Environment Setup

In [1]:
# Import required packages
import os
from openai import OpenAI
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    HostedAgentDefinition,
    ProtocolVersionRecord, 
    AgentProtocol
)
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

In [2]:
# Set environment variables
PROJECT_ENDPOINT = os.environ.get("AZURE_FOUNDRY_PROJECT_ENDPOINT")
MODEL_DEPLOYMENT = os.environ.get("AZURE_FOUNDRY_GPT_MODEL")
AGENT_NAME = "msft-docs-agent-v2"
API_VERSION = "2025-11-15-preview"

CONTAINER_IMAGE = "lazizacr.azurecr.io/mslearn-mcp-hostedagentv2:v1"
CPU = "1"
MEMORY = "2Gi"

print("Configuration loaded")

Configuration loaded


In [3]:
# Initialise Azure AI Foundry's project client
client = AIProjectClient(
    endpoint = PROJECT_ENDPOINT,
    credential = DefaultAzureCredential()
)

print("Client initialised")

Client initialised


In [4]:
# Create new hosted agent version
agent = client.agents.create_version(
    agent_name = AGENT_NAME,
    definition = HostedAgentDefinition(
        container_protocol_versions = [
            ProtocolVersionRecord(
                protocol = AgentProtocol.RESPONSES,
                version = "v1"
            )
        ],
        cpu = CPU,
        memory = MEMORY,
        image = CONTAINER_IMAGE,
        environment_variables = {
            "AZURE_FOUNDRY_PROJECT_ENDPOINT": PROJECT_ENDPOINT,
            "AZURE_FOUNDRY_GPT_MODEL": MODEL_DEPLOYMENT
        }
    ),
    foundry_features = "HostedAgents=V1Preview"
)

print(f"Agent created successfully!")
print(f"  - Agent Name: {agent.name}")
print(f"  - Version: {agent.version}")

Agent created successfully!
  - Agent Name: msft-docs-agent-v2
  - Version: 1


### Start Agent deployment in AI Foundry portal:

Option 1 - Azure Portal:
- Go to: https://ai.azure.com, open your Foundry agent and click `Start agent deployment` button.

Option 2 - Azure CLI:
- Use this CLI command: `az cognitiveservices agent start --account-name {FOUNDRY_ACCOUNT_NAME} --project-name {FOUNDRY_PROJECT_NAME} --name {AGENT_NAME} --agent-version {AGENT_VERSION}`

### Verify Agent's Availability

In [ ]:
# Test deployed agent
AGENT_ENDPOINT = f"{PROJECT_ENDPOINT}/applications/{AGENT_NAME}/protocols/openai"
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://ai.azure.com/.default"
)

openai_client = OpenAI(
    api_key=token_provider,
    base_url=AGENT_ENDPOINT,
    default_query={"api-version": API_VERSION}
)

user_message = "How I can create an Azure Function in Python?"
response = openai_client.responses.create(input=user_message)

print(f"User question: {user_message}")
print(f"Agent response: {response.output_text}")

### Housekeeping

In [6]:
# Show all agents and their versions
agents = client.agents.list()
for agent in agents:
    print(f"Agent: {agent.name}")
    versions = client.agents.list_versions(agent_name=agent.name)
    for v in versions:
        print(f"  - Version: {v.version}")

Agent: msft-docs-agent-v2
  - Version: 1


In [ ]:
# Delete specific agent version
try:
    client.agents.delete_version(
        agent_name = AGENT_NAME,
        agent_version = 1
    )
    print(f"Deleted {AGENT_NAME}:1")
    
except Exception as e:
    print(f"Error: {e}")

print("Cleanup complete")

Cleanup complete
